In [1]:
import os
if "notebooks" in os.getcwd(): os.chdir('..')

In [2]:
import pandas as pd
import numpy as np
import xarray as xr
import numpy.random as npr
import networkx as nx 

import warnings
import time
import copy
import itertools
from itertools import product as iterprod
from functools import partial
from pathlib import Path
from tqdm.notebook import tqdm
from tqdm.contrib.concurrent import process_map
import yaml

import socialbandit as sb


In [3]:
import matplotlib as mpl
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib import rcParams, font_manager

rcParams.update(mpl.rcParamsDefault)
plt.style.use('default')

# use one of the available styles 
plt.style.use('seaborn-notebook')

# further customization
rcParams['font.family'] = 'FreeSans'
rcParams['font.size'] = 12

rcParams['axes.titlesize'] = 14
rcParams['axes.labelsize'] = 12
rcParams['legend.fontsize'] = 12
rcParams['xtick.labelsize'] = 12
rcParams['ytick.labelsize'] = 12
rcParams['axes.linewidth'] = 1.5

rcParams['mathtext.fontset'] = 'cm' 
rcParams['mathtext.rm'] = 'serif'


In [4]:
def is_prevchoice_diff(A_prev, A_curr):
    # A_*: binary matrices of size "num_tasks x num_agents"
    # A_prev could also be cumulative previous A
    return np.sum(A_prev * A_curr, axis=0) < 1.0
    
def get_task_choices(A):
    inds = np.where(A > 0)
    return inds[0][np.argsort(inds[1])]

def get_task_rewards(Y):
    return Y.sum(axis=0)

def aggregate_rewards(rewards, condition):
    num = np.mean(condition.sum(axis=0))
    mag = np.mean(np.abs(rewards[condition])) if num > 0 else 0.0
    return dict(num=num, mag=mag)

def calculate_entropy(X):
    _, n = np.unique(X, return_counts=True)
    P = n/sum(n)
    return np.sum(-P * np.log2(P))

In [5]:
class PseudoAverageCumTime:
    # this is a pseduo class to create a function for time average with a persistent time variable
    # needed for when using a function with only one argument and no internal place for time update
    _t = -1
    
    def __init__(self):
        self._t = 0
    
    def __call__(self, X):
        self._t += 1
        return X / self._t


In [6]:
class SocialMAB_ChildDev(sb.models.SocialMultiArmedBandit):
    def __init__(self, 
                 num_agents=20, 
                 max_time=1200,
                 social_content = 'belief', 
                 social_network = None,
                 gamma = 1,
                 mu0 = 100.0, 
                 var0 = 40.0, 
                 softmax_tau = 1.0, 
                 BMT_error = 'use-tasks', 
                 agg_explore_step=50, 
                 agg_reward_step=400):
        
        if type(gamma) is list:
            if len(gamma) != 2: 
                raise ValueError("`gamma` can only either be a 2-element list or a scalar")
        else:
            if gamma < 0 or gamma > 1:
                raise ValueError("`gamma` needs to be in [0, 1] if it's a scalar input")
            gamma = [gamma, 1.0 - gamma]
            
        self.gamma_acl = gamma[0] # for action learning scaling
        self.gamma_sol = gamma[1] # for social learning scaling
        
        # Social content for social influence
        social_content_opts = {
            'belief': 'M',
            'action': 'A',
            'reward': 'Y', 
            'cum_reward': 'Yc',
            'mean_reward': ('Yc', PseudoAverageCumTime())
        }
        
        social_content = social_content.lower()
        if social_content not in social_content_opts: 
            raise ValueError("`social_content` cannot be {social_content}. Available options are {list(social_content_opts.keys())}.")

        content_fn = sb.social.ContentConstructor(*social_content_opts[social_content])
        
        # Social network constructor/functions
        social_fn = None
        if social_network is not None:                
            social_fn = sb.social.StaticSocialNetwork(W = social_network)
        
        # Social setting constructur from social content and network        
        social_settings = sb.social.SocialSetting(
            N = num_agents, 
            social_fn = social_fn,
            content_fn = content_fn
        )
        
        # Social learner constructor
        social_learner = None
        if social_network is not None:
            social_learner = sb.learners.MeanFriendContentLearner()
        
        # Childhood development settings
        task_settings = sb.tasks.ChildDevelopmentEnvironment(env_levels = 12)
        
        # Initializer
        initializer = sb.initializers.InitializerBanditAgnostic(
            mu_fn = lambda x: mu0,
            sigma2_fn = lambda x: var0
        )
        
        # Sampler
        action_sampler = sb.tasks.SoftmaxSampler(tau = softmax_tau)
        
        # Bayesian mean tracker to update belief
        if type(BMT_error) is str:
            if BMT_error.lower() == 'use-tasks':
                bmt_var_err = task_settings
            else: 
                try:
                    bmt_var_err = float(BMT_error)
                except:
                    raise ValueError("If `BMT_error` is a string, only 'use-tasks' or a string of a float is accepted at this point")
        elif type(BMT_error) in [float, int]:
            bmt_var_err = BMT_error
        else:
            raise ValueError("`BMT_error` can either be a str/float/int")
        
        belief_updater  = sb.beliefs.BayesianMeanTracker(var_error = bmt_var_err)
        
        # Initialize 
        super().__init__(
            task_settings   = task_settings,
            social_settings = social_settings,
            clock           = sb.utils.ExperimentClock(T = max_time),
            initializer     = initializer,
            action_sampler  = action_sampler,
            belief_updater  = belief_updater,
            action_learner  = sb.learners.MeanGreedyExploit(),
            social_learner  = social_learner
        )
        
        self.set_state_managers()
        
        # Set up analyses variables
        analysis_size = (max_time+1, num_agents)
        cum_choice_size = (self.num_tasks, num_agents)
        self.analysis = dict(
            params = dict(
                agg_explore_step = agg_explore_step, 
                agg_reward_step = agg_reward_step
            ),
            per_agent = dict(
                cum_choice = np.zeros(cum_choice_size),
                explore = np.zeros(analysis_size),
                choice = np.zeros(analysis_size),
                reward = np.zeros(analysis_size)
            ),
            time = dict(
                explore = range(0, max_time, agg_explore_step),
                reward = np.array(range(0, max_time, agg_reward_step)) + agg_reward_step,
            ),
            aggregate = dict(
                explore_num = [],
                unq_choices = [],
                explore_ent = [], 
                mean_reward = [],
                loss_num = [],
                loss_mag = [],
                gain_num = [],
                gain_mag = []                
            ),
            aux = dict(
                Q_acl = np.zeros(max_time+1), 
                Q_sol = np.zeros(max_time+1)
            )
        )

    def update_utility(self):
        Q_acl = self.gamma_acl * self.learn_action()
        Q_sol = self.gamma_sol * self.learn_social()
        self.states.Q = Q_acl + Q_sol

    def analyze(self):
        t = self.t
        
        # auxilary 
        aux_var = self.analysis['aux']
        aux_var['Q_acl'][t] = np.mean(self.states.Q_acl)
        aux_var['Q_sol'][t] = np.mean(self.states.Q_sol)
        
        # analysis
        anly_per_agent = self.analysis['per_agent']
        anly_params = self.analysis['params']
        agg_anly = self.analysis['aggregate']
        
        anly_per_agent['explore'][t,:] = is_prevchoice_diff(anly_per_agent['cum_choice'], self.states.A)
        anly_per_agent['cum_choice'] += self.states.A
        
        anly_per_agent['choice'][t,:] = get_task_choices(self.states.A)
        anly_per_agent['reward'][t,:] = get_task_rewards(self.states.Y)
        
        agg_explore_step = anly_params['agg_explore_step']
        if t % agg_explore_step == 0 and t > 1:
            agg_anly['explore_num'].append(
                np.mean(anly_per_agent['explore'][t-agg_explore_step:t,:].sum(axis=0))        
            )
            
            # choice_matrix = anly_per_agent['choice'][t-agg_explore_step:t,:]
            choice_matrix = anly_per_agent['choice'][:t,:]
            agg_anly['explore_ent'].append(
                np.mean([calculate_entropy(X) for X in choice_matrix.T])                
            )
            
            agg_anly['unq_choices'].append(
                np.mean([len(np.unique(X)) for X in choice_matrix.T])     
            )
            
            
            
        agg_reward_step = anly_params['agg_reward_step']
        if t % agg_reward_step == 0 and t > 0:
            reward_matrix = anly_per_agent['reward'][t-agg_reward_step:t]
            agg_anly['mean_reward'].append(np.mean(reward_matrix))
            agg_loss = aggregate_rewards(reward_matrix, reward_matrix < 0)
            
            agg_anly['loss_num'].append(agg_loss['num'])
            agg_anly['loss_mag'].append(agg_loss['mag'])
            
            agg_gain = aggregate_rewards(reward_matrix, reward_matrix > 0)
            
            agg_anly['gain_num'].append(agg_gain['num'])
            agg_anly['gain_mag'].append(agg_gain['mag'])
            
        if t == self.clock.T:
            for k, v in agg_anly.items():
                agg_anly[k] = np.array(v)


In [7]:
num_agents = 50

agent_social_network = nx.to_numpy_array(nx.erdos_renyi_graph(num_agents, p = 0.9))
# agent_social_network = nx.to_numpy_array(nx.barabasi_albert_graph(num_agents, m=5))
# agent_social_network = None 

model = SocialMAB_ChildDev(
    num_agents = num_agents,
    social_content = 'belief', 
    social_network = agent_social_network,
    gamma = 0.5,
    BMT_error = '3600',
)

model.run(tqdm_fn = tqdm)
model.analysis['aggregate']

  0%|          | 0/1200 [00:00<?, ?it/s]

{'explore_num': array([3.024e+01, 5.620e+00, 1.400e-01, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 4.036e+01, 2.866e+01, 1.682e+01, 9.020e+00,
        5.300e+00, 2.780e+00, 1.920e+00, 1.240e+00, 5.600e-01, 3.400e-01,
        3.400e-01, 2.400e-01, 4.000e-02, 8.000e-02, 6.000e-02, 6.000e-02]),
 'unq_choices': array([ 31.24,  36.86,  37.  ,  37.  ,  37.  ,  37.  ,  37.  ,  37.  ,
         76.98, 105.46, 122.1 , 131.02, 136.28, 139.  , 140.88, 142.1 ,
        142.66, 143.  , 143.34, 143.58, 143.62, 143.7 , 143.76, 143.82]),
 'explore_ent': array([4.81734981, 5.06035225, 5.09814353, 5.10198334, 5.09928117,
        5.09800184, 5.09393596, 5.08952144, 5.58755828, 5.96447655,
        6.22444775, 6.39619824, 6.50953981, 6.58217633, 6.63066547,
        6.65310997, 6.65773001, 6.64645638, 6.62557579, 6.59856009,
        6.5699984 , 6.53955454, 6.50698561, 6.47351491]),
 'mean_reward': array([10.23428033, 43.81346782, 76.95531133]),
 'loss_num': array([152.22, 110.68,  50.94]),

In [12]:
def create_variations(d, add_id=True, num_trials=1):
    keys = list(d.keys())
    values = list(d.values())
    
    if not add_id:
        df = pd.DataFrame(itertools.product(*values), columns=keys)
        return df.to_dict('records') * num_trials
    
    trial_ids = list(range(num_trials))
    
    combos = list(itertools.product(*values)) # first, combinations of values
    combos = [list(x) + [i] for i, x in enumerate(combos)] # then, add a variation ID
    combos = list(itertools.product(combos, trial_ids)) # duplicating with trials
    combos = [list(x[0]) + [x[1], i] for i, x in enumerate(combos)] # lastly, add unique ID
    
    df = pd.DataFrame(combos, columns=keys + ['_var_id', '_trial_id', '_unq_id'])

    return df.to_dict('records')

def experiment_ER(num_agents, social_content, p_social, utility_gamma, BMT_error, mu0=100.0, *args, **kwargs):
    
    social_network = None
    if p_social > 0:
        social_network = nx.erdos_renyi_graph(num_agents, p = p_social)
        social_network = nx.to_numpy_array(social_network)
    
    model = SocialMAB_ChildDev(
        num_agents = num_agents,
        social_content = social_content, 
        social_network = social_network,
        gamma = utility_gamma,
        BMT_error = BMT_error,
        mu0 = mu0
    )

    model.run(tqdm_fn=None)
    
    result_keys = dict(
        explore = ['explore_num', 'unq_choices', 'explore_ent'],
        reward = ['mean_reward', 'loss_num', 'loss_mag', 'gain_num', 'gain_mag']
    )
    
    agg_anly = model.analysis['aggregate']
    agg_time = model.analysis['time']
    
    results = {}
    for section_key, agg_keys in result_keys.items():
        results[section_key] = {k: v for k, v in agg_anly.items() 
                                if k in agg_keys}
        results[section_key].update({'time': agg_time[section_key]})
        results[section_key] = pd.DataFrame(results[section_key])
    
    return results

def save_results(file_prefix, results, variations, var_dict):
    results = {k: pd.concat([x[k] for x in results], ignore_index=True)
               for k in results[0].keys()}
    results.update({'variations': pd.DataFrame(variations)})
    
    for k, df in results.items(): 
        file_name = f'{file_prefix}-{k}.parq'
        df.to_parquet(file_name, engine='fastparquet')
    
    with open(f'{file_prefix}-variations.yaml', 'w') as f: 
        yaml.safe_dump(var_dict, f)
    
    return results

def _run_1_exp(variation, fn):
    dfs = fn(**variation)
    dfs = {k: df.assign(**variation) for k, df in dfs.items()}    
    return dfs

def run_experiment(fn, var_dict, file_prefix, num_trials=1, 
                   max_workers=2, chunksize=1):      
    variations = create_variations(var_dict, num_trials=num_trials)

    results = process_map(
        partial(_run_1_exp, fn=fn),
        variations, 
        max_workers=max_workers,
        chunksize=chunksize
    )

    results = save_results(file_prefix, results, variations, var_dict)
    
    return results

In [13]:
# Testing
var_dict = dict(
    num_agents = [20],
    social_content = ['belief'],
    p_social = [0, 0.5, 1.0],
    utility_gamma = [0, 0.5, 1.0],
    BMT_error = [3600]
)

results = run_experiment(
    fn = experiment_ER,
    var_dict = var_dict,
    file_prefix = 'data/output/test/test1', 
    num_trials = 3,
    max_workers = 5
)

  0%|          | 0/27 [00:00<?, ?it/s]

In [ ]:
# Run
var_dict = dict(
    num_agents = [100],
    social_content = ['belief',  'reward'],
    p_social = [0, 0.2, 0.4, 0.6, 0.8, 1.0],
    utility_gamma = [0, 0.2, 0.4, 0.6, 0.8, 1.0],
    mu0 = [25.0, 100.0], 
    BMT_error = ['3600', 'use-tasks']
)

results = run_experiment(
    fn = experiment_ER,
    var_dict = var_dict,
    file_prefix = 'data/output/test/meansocial_childdev', 
    num_trials = 10,
    max_workers = 6,
    chunksize = 6
)


  0%|          | 0/2880 [00:00<?, ?it/s]